# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [5]:
import numpy as np
import pandas as pd
import requests 
import xlsxwriter
import math 
import json

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [6]:
stocks = pd.read_csv("sp_500_stocks.csv")
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [7]:
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [15]:
symbol = "AAPL"
api_url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}"
try:
    data = requests.get(api_url).json()
except ValueError:
    print("Response content is not valid JSON")
print(data)

{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': 'BDAN)AQCLSML S  AEENGOAKGT/TSRLE(', 'calculationPrice': 'close', 'open': 128.01, 'openTime': 1675270475841, 'openSource': 'iiclfofa', 'close': 124.72, 'closeTime': 1622488168334, 'closeSource': 'ififolca', 'high': 125.051, 'highTime': 1624129057649, 'highSource': 'tneeemudirc ld 1ipe5 ay', 'low': 125.38, 'lowTime': 1661257372254, 'lowSource': 'eu ep1 dnr5iydticemel a', 'latestPrice': 123.58, 'latestSource': 'Close', 'latestTime': 'December 4, 2020', 'latestUpdate': 1685405270289, 'latestVolume': 79299720, 'iexRealtimePrice': None, 'iexRealtimeSize': None, 'iexLastUpdated': None, 'delayedPrice': 126.796, 'delayedPriceTime': 1663072749743, 'oddLotDelayedPrice': 124.6, 'oddLotDelayedPriceTime': 1644523552812, 'extendedPrice': 123.86, 'extendedChange': -0.11, 'extendedChangePercent': -0.0009, 'extendedPriceTime': 1664383151889, 'previousClose': 124.98, 'previousVolume': 79848496, 'change': -0.71, 'changePercent': -0.00579, 

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [19]:
price = data["latestPrice"]
market_cap = data["marketCap"]

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [27]:
my_columns = ["Ticker", "Stock Price", "Market Capitalization", "Number of Shares to Buy"]
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,0,0,0,0


In [33]:
final_dataframe.append(
pd.Series(
[
    symbol,
    price,
    market_cap,
    "N/A"
],
    index = my_columns
),
    
    ignore_index=True
)

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,0,0.00,0,0
1,AAPL,123.58,2122994536969,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [54]:
final_dataframe = pd.DataFrame(columns=my_columns)
for stock in stocks["Ticker"][:50]:
    api_url = f"https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
    pd.Series([
        stock,
        data["latestPrice"],
        data["marketCap"],
        "N/A"
    ],
        index = my_columns
    ),
        ignore_index = True
    )

In [55]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,117.87,35746522137,N/A
1,AAL,16.60,10042788429,N/A
2,AAP,154.86,10549795872,N/A
3,AAPL,126.21,2120168550988,N/A
4,ABBV,110.22,198376786584,N/A
5,ABC,105.62,21306910914,N/A
6,ABMD,277.47,12551354486,N/A
7,ABT,111.30,194400933355,N/A
8,ACN,253.91,174277708034,N/A
9,ADBE,494.00,240215139374,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [65]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [74]:
symbol_groups = list(chunks(stocks["Ticker"], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])
final_dataframe = pd.DataFrame(columns = my_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f"https://sandbox.iexapis.com/stable /stock/market/batch?symbols=aapl,fb,tsla&types=quote,news,chart&range=1m&last=5"
    

SyntaxError: unexpected EOF while parsing (<ipython-input-74-d41b3e066d98>, line 7)

## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

## Saving Our Excel Output

Saving our Excel file is very easy: